#### Contents:
1. [Background](#Background)
2. [Environment Setup](#Environment-Setup)
    - [Install the required python packages](#Install-the-required-python-packages)
    - [Import the required libraries](#Import-the-required-libraries)
3. [Download and prepare the data](#Download-and-prepare-the-data)
4. [Scenario 1: Related time series are available in the forecast horizon](#Scenario-1:-Related-time-series-are-available-in-the-forecast-horizon)
5. [Scenario 2: Related time series is not available in the forecast horizon](#Scenario-2:-Related-time-series-is-not-available-in-the-forecast-horizon)
6. [Scenario 3: Model all the time series as target series](#Model-all-the-time-series-as-target-series)

## 1. Background

Multivariate time series forecasting is a common problem and more recently deep learning models have been applied to time series forecasting. [GluonTS](https://ts.gluon.ai/index.html) is a deep learning toolkit for probabilistic modelling of time series. This notebook shows you different ways in which one can model a multivariate time series problem (time series with related variables) using different models that are implemented in GluonTS.
The following models are explored in this notebook -
- [DeepAR](https://ts.gluon.ai/api/gluonts/gluonts.model.deepar.html)
- [Transformer](https://ts.gluon.ai/api/gluonts/gluonts.model.transformer.html)
- [MQ-CNN](https://ts.gluon.ai/api/gluonts/gluonts.model.seq2seq.html)
- [Temporal Fusion Transformer](https://ts.gluon.ai/api/gluonts/gluonts.model.tft.html)
- [LSTNet](https://ts.gluon.ai/api/gluonts/gluonts.model.lstnet.html)

## 2. Environment Setup

Please run this notebook on an instance that has a GPU. (p2.xlarge or higher)  

## 2.1 Install the required python packages

In [ ]:
!pip uninstall -y mxnet

In [ ]:
!pip install --upgrade mxnet~=1.7
!pip install gluonts
!pip install mxnet-cu102==1.7.0

## 2.2 Import the required libraries

In [ ]:
import pandas as pd
from gluonts.dataset.common import (
    CategoricalFeatureInfo,
    ListDataset,
    MetaData,
    TrainDatasets,
    load_datasets
)

from gluonts.dataset.field_names import FieldName
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.transformer import TransformerEstimator
from gluonts.model.lstnet import LSTNetEstimator
from gluonts.model.seq2seq import MQCNNEstimator
from gluonts.model.seq2seq import MQRNNEstimator
from gluonts.model.tft import TemporalFusionTransformerEstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator, MultivariateEvaluator
from gluonts.mx.trainer import Trainer
from gluonts.dataset.multivariate_grouper import MultivariateGrouper

import mxnet as mx

from itertools import islice
import matplotlib.pyplot as plt
%matplotlib inline

## 3. Download and prepare the data

We use the [PM 2.5 dataset](https://archive.ics.uci.edu/ml/datasets/Beijing+PM2.5+Data) from the UCI Machine Learning Repository.

The dataset contains PM 2.5 data from the US Embassy in Beijing and is supplemented with meteorological data. The following columns are part of the data - 
- No: row number
- year: year of data in this row
- month: month of data in this row
- day: day of data in this row
- hour: hour of data in this row
- pm2.5: PM2.5 concentration (ug/m^3)
- DEWP: Dew Point (â„ƒ)
- TEMP: Temperature (â„ƒ)
- PRES: Pressure (hPa)
- cbwd: Combined wind direction
- Iws: Cumulated wind speed (m/s)
- Is: Cumulated hours of snow
- Ir: Cumulated hours of rain

Given the above information, here is how the different features in the dataset are treated - 
- pm2.5 is the target variable. 
- Meteorological variables like 'TEMP', 'DEWP' and 'PRES' can be treated as related time series with real values.
- 'cbwd' is a categorical variable and varies with time and can be treated as a dynamic categorical feature.

There are different ways in which one can model multivariate time series problems depending on the availability of related time series features in the forecast horizon. This notebook illustrates them by assuming the presence or absence of the meteorological variables in the forecast horizon.

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00381/PRSA_data_2010.1.1-2014.12.31.csv

In [ ]:
df = pd.read_csv("PRSA_data_2010.1.1-2014.12.31.csv")

#combine month,day,hour into a timestamp
df['Timestamp'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])

#set an ID to identify a time series
df['id'] = 0

#set the type of the categorical variable
df["cbwd"] = df["cbwd"].astype('category')
df["cbwd_cat"] = df["cbwd"].cat.codes

In [ ]:
df.columns

## 4. Related time series are available in the forecast horizon

In this section, you will assume that the meteorological variables (TEMP, DEWP, PRES) are available to the model in the forecast horizon. In real life, this could be from a weather prediction model or forecast.

The following cells compare a DeepAR and Transformer in this particular scenario.

### 4.1 Prepare the training and testing dataset

In [ ]:
forecast_length = 120
num_backtest_windows = 2

In [ ]:
training_data_list = []
test_data_list = []

for i in reversed(range(1, num_backtest_windows+1)):
      
    training_data = [
        {
            "start": df.iloc[0]["Timestamp"],
            "target": df["pm2.5"][:-forecast_length*i],
            "feat_static_cat": [0],
            "feat_dynamic_real": [df["TEMP"][:-forecast_length*i],
                                 df["DEWP"][:-forecast_length*i]],
            "feat_dynamic_cat": [df["cbwd_cat"][:-forecast_length*i]]
        }
        ]
    
    # create testing data.
    test_data = [
        {
            "start": df.iloc[0]["Timestamp"],
            "target": df["pm2.5"][:-forecast_length*(i-1)] if i>1 else df["pm2.5"][:],
            "feat_static_cat": [0],
            "feat_dynamic_real": [df["TEMP"][:-forecast_length*(i-1)] if i>1 else df["TEMP"][:],
                                 df["DEWP"][:-forecast_length*(i-1)] if i>1 else df["DEWP"][:]],
            "feat_dynamic_cat": [df["cbwd_cat"][:-forecast_length*(i-1)] if i>1 else df["cbwd_cat"][:]]
        }
        ]

    training_data_list.append(ListDataset(training_data, freq='1h'))
    test_data_list.append(ListDataset(test_data, freq='1h'))

In [ ]:
#function that takes a gluonTS estimator, trains the model and predicts it for every pair of train and test dataset
def backtest(model,
             training_data_list,
             test_data_list,
             num_backtest_windows):
    
    forecasts = []
    obs = []
    
    #train a model for every backtest window
    for i in range(num_backtest_windows):
        predictor = model.train(training_data_list[i],
                               force_reinit=True)
        forecast_it, ts_it = make_evaluation_predictions(test_data_list[i], 
                                                         predictor=predictor, 
                                                         num_samples=100)
        forecasts.extend(list(forecast_it))
        obs.extend(list(ts_it))
    return forecasts, obs

####  DeepAR

In [ ]:
deepar = DeepAREstimator(freq="1h",
                         use_feat_static_cat=True,
                         use_feat_dynamic_real=True,
                         cardinality=[1],
                         prediction_length=forecast_length,
                         trainer=Trainer(epochs=30, ctx = mx.context.gpu(0)),
                         num_cells=40)
forecast_deepar, obs_deepar = backtest(deepar,
                                       training_data_list,
                                       test_data_list,
                                       num_backtest_windows)

#### Transformer

In [ ]:
transformer = TransformerEstimator(freq="1h",
                                   use_feat_dynamic_real=True,
                                   #context_length=168,
                                   prediction_length=forecast_length,
                                   trainer=Trainer(epochs=10,
                                                   learning_rate=0.01,
                                                   ctx = mx.context.gpu(0))
                                  )
forecast_transformer, obs_transformer = backtest(transformer,
                                                 training_data_list,
                                                 test_data_list,
                                                 num_backtest_windows)

#### Evaluation

In [ ]:
evaluator = Evaluator(quantiles=[0.5], seasonality=None)
agg_metrics_deepar, item_metrics_deepar = evaluator(iter(obs_deepar), 
                                                        iter(forecast_deepar), 
                                                        num_series=len(forecast_deepar))

evaluator = Evaluator(quantiles=[0.5], seasonality=None)
agg_metrics_transformer, item_metrics_transformer = evaluator(iter(obs_transformer),
                                                              iter(forecast_transformer),
                                                              num_series=len(forecast_transformer))


#### Results

In [ ]:
pd.concat([pd.DataFrame.from_dict(agg_metrics_deepar, orient='index').rename(columns={0: "DeepAR"}),
           pd.DataFrame.from_dict(agg_metrics_transformer, orient='index').rename(columns={0: "Transformer"})],
         axis=1)

In [ ]:
def plot_forecasts(obs, forecasts, past_length, start, stop, step, title):
    for target, forecast in zip(obs, forecasts):
        ax = target[-past_length:].plot(figsize=(12, 5), linewidth=2)
        forecast.plot(color='g')
        plt.ylabel('PM2.5 concentration (ug/m^3)')
        plt.title(title)
        plt.grid(which='both')
        plt.legend(["observations", "median prediction", "90% confidence interval", "50% confidence interval"])
        plt.show()

The below charts plot the observed PM2.5 against the forecast from DeepAR and Transformer. Since, the model computes probabilistic forecasts, it is possible to draw a confidence interval around the median prediction. These charts show a 50% and 90% confidence interval.

#### Plot Sample Forecast - DeepAR

In [ ]:
plot_forecasts(obs_deepar, forecast_deepar, 340, 0, 2, 1, 'deepar')

#### Plot Sample Forecast - Transformer

In [ ]:
plot_forecasts(obs_transformer, forecast_transformer, 340, 0, 2, 1, 'transformer')

## 5. Related time series is not available in the forecast horizon

In this section, you will see how to train models when the related time series (meteorological features in this case) is not available in the forecast horizon. The meteorological variables are only present for the historical time period and can hence be used for training the model.

Seq2Seq models like [MQ-CNN](https://ts.gluon.ai/api/gluonts/gluonts.model.seq2seq.html) which uses a CNN as an encoder and a MLP as a decoder can be used in this scenario.

[Temporal Fusion Transformer](https://arxiv.org/abs/1912.09363) is another architecture that combines recurrent layers and attention layers to enable the usage of a mix of inputs like exogenous variables that are only observed historically and other static and dynamic covariates.

We compare the above to models in this section.

### 5.1 Prepare the training and testing dataset to use 'past_feat_dynamic_real'

In [ ]:
training_data_list = []
test_data_list = []

for i in reversed(range(1, 3)):
      
    training_data = [
        {
            "start": df.iloc[0]["Timestamp"],
            "target": df["pm2.5"][:-forecast_length*i],
            "past_feat_dynamic_real": [df["TEMP"][:-forecast_length*i],
                                 df["DEWP"][:-forecast_length*i]],
        }
        ]
    
    # create testing data.
    test_data = [
        {
            "start": df.iloc[0]["Timestamp"],
            "target": df["pm2.5"][:-forecast_length*(i-1)] if i>1 else df["pm2.5"][:],
            "past_feat_dynamic_real": [df["TEMP"][:-forecast_length*(i-1)] if i>1 else df["TEMP"][:],
                                 df["DEWP"][:-forecast_length*(i-1)] if i>1 else df["DEWP"][:]],
        }
        ]

    training_data_list.append(ListDataset(training_data, freq='1h'))
    test_data_list.append(ListDataset(test_data, freq='1h'))

#### MQ-CNN

In [ ]:
#At times, one can encounter exploding gradients and as a result the loss can become a NaN. 
#set hybridize=False. May be related to https://github.com/awslabs/gluon-ts/issues/833
mqcnn = MQCNNEstimator(freq="1h",
                       use_past_feat_dynamic_real=True,
                       prediction_length=forecast_length,
                       trainer=Trainer(epochs=30,
                                       learning_rate=0.001,
                                       #clip_gradient=3,
                                       #batch_size=32,
                                       #num_batches_per_epoch=16,
                                       hybridize=False,
                                       ctx = mx.context.gpu(0)),
                       
                      )
forecast_mqcnn, obs_mqcnn = backtest(mqcnn,
                                   training_data_list,
                                   test_data_list,
                                   num_backtest_windows)

#### Temporal Fusion Transformer

In [ ]:
training_data_list = []
test_data_list = []

for i in reversed(range(1, 3)):
      
    training_data = [
        {
            "start": df.iloc[0]["Timestamp"],
            "target": df["pm2.5"][:-forecast_length*i],
            "past_feat_dynamic_real_1": df["TEMP"][:-forecast_length*i],
            "past_feat_dynamic_real_2": df["DEWP"][:-forecast_length*i],
            "past_feat_dynamic_real_3": df["Ir"][:-forecast_length*i]
        }
        ]
    
    # create testing data.
    test_data = [
        {
            "start": df.iloc[0]["Timestamp"],
            "target": df["pm2.5"][:-forecast_length*(i-1)] if i>1 else df["pm2.5"][:],
            "past_feat_dynamic_real_1": df["TEMP"][:-forecast_length*(i-1)] if i>1 else df["TEMP"][:],
            "past_feat_dynamic_real_2": df["DEWP"][:-forecast_length*(i-1)] if i>1 else df["DEWP"][:],
            "past_feat_dynamic_real_3": df["Ir"][:-forecast_length*(i-1)] if i>1 else df["Ir"][:]
        }
        ]

    training_data_list.append(ListDataset(training_data, freq='1h'))
    test_data_list.append(ListDataset(test_data, freq='1h'))

feat_past_dynamic_real = ["past_feat_dynamic_real_1", "past_feat_dynamic_real_2", "past_feat_dynamic_real_3"]

In [ ]:
#https://github.com/awslabs/gluon-ts/issues/1075
tft = TemporalFusionTransformerEstimator(freq = '1h',
                                         context_length=168,
                                         prediction_length = forecast_length,
                                         trainer=Trainer(epochs=30,
                                                         learning_rate=0.001,
                                                         ctx = mx.context.gpu(0)),
                                         hidden_dim=32,
                                         variable_dim=8,
                                         num_heads=4,
                                         num_outputs=3,
                                         num_instance_per_series=100,
                                         dropout_rate=0.1,
                                         dynamic_feature_dims={
                                             'past_feat_dynamic_real_1': 1,
                                             'past_feat_dynamic_real_2': 1,
                                             'past_feat_dynamic_real_3': 1
                                         }, # dimensions of dynamic real features
                                         past_dynamic_features=feat_past_dynamic_real,
                                        )
forecast_tft, obs_tft = backtest(tft,
                                 training_data_list,
                                 test_data_list,
                                 num_backtest_windows)

#### Evaluation

In [ ]:
evaluator = Evaluator(quantiles=[0.5], seasonality=None)
agg_metrics_mqcnn, item_metrics_mqcnn = evaluator(iter(obs_mqcnn), 
                                                    iter(forecast_mqcnn), 
                                                    num_series=len(forecast_mqcnn))

evaluator = Evaluator(quantiles=[0.5], seasonality=None)
agg_metrics_tft, item_metrics_tft = evaluator(iter(obs_tft),
                                                  iter(forecast_tft),
                                                  num_series=len(forecast_tft))


#### Results

In [ ]:
pd.concat([pd.DataFrame.from_dict(agg_metrics_mqcnn, orient='index').rename(columns={0: "MQ-CNN"}),
           pd.DataFrame.from_dict(agg_metrics_tft, orient='index').rename(columns={0: "TFT"})],
         axis=1)

In [ ]:
# 'QuantileForecast.plot' plots all the quantiles as line plots.
# This is some boiler plate code to plot an interval around the median 
# using the 10th and 90th quantile
def plot_from_quantile_forecast(obs, past_length, lower_bound, upper_bound, forecasts):
    plt.figure(figsize=(12,6))
    plt.plot(obs[0][-forecast_length-past_length:], label='observed')
    plt.plot(obs[0][-forecast_length:].index, 
             lower_bound,
            color='g',
            alpha=0.3,
            label='10th quantile')
    plt.plot(obs[0][-forecast_length:].index, 
             forecasts,
            color='g',
            label='median prediction')
    plt.plot(obs[0][-forecast_length:].index, 
             upper_bound,
            color='g',
            alpha=0.3,
            label='90th quantile')
    plt.fill_between(obs[0][-forecast_length:].index,
                     lower_bound, 
                     upper_bound,
                    color='g',
                    alpha=0.3)
    plt.ylabel('PM2.5 concentration (ug/m^3)')
    
    plt.legend()
    plt.grid(which="both")
    plt.show()

The two models illustrated in this section forecast quantiles. Hence to construct an interval, one needs to pick forecasts at different quantiles. The charts below use the 10th and the 90th quantile forecast to construct an interval.

#### Plot Sample Forecast - MQCNN

In [ ]:
plot_from_quantile_forecast(obs_mqcnn, 
                            100, 
                            forecast_mqcnn[0].forecast_array[0], 
                            forecast_mqcnn[0].forecast_array[8],
                            forecast_mqcnn[0].forecast_array[4])

#### Plot Sample Forecast - Temporal Fusion Transformer (TFT)

In [ ]:
plot_from_quantile_forecast(obs_tft, 
                            100, 
                            forecast_tft[0].forecast_array[1], 
                            forecast_tft[0].forecast_array[2],
                            forecast_tft[0].forecast_array[0])

## 6. Model all the time series as target series

In this case, we forecast pm2.5 and the other meteorological features together as multivariate variables.

Models like [LSTNet](https://ts.gluon.ai/api/gluonts/gluonts.model.lstnet.html) allow one to treat all the related time series in a multivariate fashion. One can train a model to forecast all the time series simultaneously.

For this, the data needs to be prepared in a different way and the below cell does that.

#### LSTNet

In [ ]:
train = df.transpose()
train2 = train.to_numpy()

target=train2[[5,6,7,8],:]

#prediction_length=24

start= [df.iloc[0]["Timestamp"] for _ in range(4)]


train_ds = ListDataset([{FieldName.TARGET: target, 
                         FieldName.START: start
                         } 
                        for (target, start) in zip(target[:, :-forecast_length], 
                                                            start)],
                      freq='1h')

test_ds = ListDataset([{FieldName.TARGET: target, 
                        FieldName.START: start
                       }
                       for (target, start) in zip(target[:, :],
                                                  start)],
                      freq='1h')


lstnet_estimator=LSTNetEstimator(freq='1h', 
                          prediction_length=forecast_length, 
                          context_length=336, 
                          num_series=4, 
                          skip_size=10, 
                          ar_window=320, 
                          channels=80, 
                          trainer = Trainer(epochs=400,
                                            ctx = mx.context.gpu(0)), 
                          dropout_rate = 0.4, 
                          output_activation = 'sigmoid', 
                          rnn_cell_type = 'gru', 
                          rnn_num_cells = 100, 
                          rnn_num_layers = 6, 
                          skip_rnn_cell_type = 'gru', 
                          skip_rnn_num_layers = 3, 
                          skip_rnn_num_cells = 10, 
                          scaling = True)


grouper_train = MultivariateGrouper(max_target_dim=4)

train_ds = grouper_train(train_ds)

lstnet_predictor = lstnet_estimator.train(train_ds)

grouper_test = MultivariateGrouper(max_target_dim=4)

test_ds = grouper_test(test_ds)

forecast_lstnet, obs_lstnet = make_evaluation_predictions(test_ds,
                                                          predictor=lstnet_predictor,
                                                         num_samples=100)

forecast_lstnet = list(forecast_lstnet)
obs_lstnet = list(obs_lstnet)

#### Evaluation

In [ ]:
evaluator = MultivariateEvaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics_lstnet, item_metrics_lstnet = evaluator(obs_lstnet, forecast_lstnet, num_series=len(test_ds))

index_series_map = {0: "pm 2.5",
                   1: "DEWP",
                   2: "TEMP",
                   3: "PRES"}
metrics_lstnet = []
for i in range(4):
    metrics = [k for k in agg_metrics_lstnet.keys() if k.startswith(str(i))]
    metrics_lstnet.append(pd.DataFrame.from_dict({m[2:]:agg_metrics_lstnet[m] for m in metrics},
                                                 orient='index').rename(columns={0: index_series_map[i]}))
pd.concat(metrics_lstnet, axis=1)  

#### Plot Sample Forecast - LSTNet

The below plots show the forecasts for each of the target time series as defined in the above cell. The results from the LSTNet model that is trained to forecast all the time series simultaneously is not great. One, probably needs to do a more thorough hyperparameter optimization. But this can be a good model to explore when one wants to build a single model to forecast multiple time series.

In [ ]:

for x, y  in zip(obs_lstnet, forecast_lstnet):
    for i in range(4):
        plt.figure(figsize=(12,6))
        plt.plot(x[i][-forecast_length-100:])
        median = y.copy_dim(i).quantile(0.5)
        y_10 = y.copy_dim(i).quantile(0.1)
        y_90 = y.copy_dim(i).quantile(0.9)
        #print(y_10)
        #print(y_90)
        plt.plot(x[i][-forecast_length:].index,
                 median,
                color='g')
        plt.fill_between(x[i][-forecast_length:].index,
                         y_10,
                        y_90,
                        color='g',
                        alpha=0.3)
        plt.title(index_series_map[i])
        plt.show()

In conclusion, this notebook illustrates how one can use different deep learning models that are defined in [GluonTS](https://ts.gluon.ai/index.html). The choice of models depends on the nature of the covariates and exogenous variables that are present in the dataset. GluonTS provides a rich variety of recent deep learning based models for modelling time series and this notebook can be used to quickly benchmark some of them so that one can shortlist a couple for further experimentation and fine-tuning.